In [1]:
import ITensors:op
import Pkg; Pkg.add("HDF5")
import Pkg; Pkg.add("Plots")
import Pkg; Pkg.add("PyFormattedStrings")
import Pkg; Pkg.add("LinearAlgebra")
import Pkg; Pkg.add("LaTeXStrings")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
using ITensors, ITensorMPS
import ITensors:op
using HDF5
using Plots
using PyFormattedStrings
using LinearAlgebra

ITensors.disable_warn_order()

14

In [3]:
include("overload_Spin=3h.jl")

In [4]:
# construction of the MPO hamiltonian
#------------------------------------------
function Hamiltonian(sites)
    ops = OpSum()

    N = length(sites)
    # spin-spin interaction
    for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)
    #--------------------------
    return H
end

Hamiltonian (generic function with 1 method)

In [5]:

#four point RDM of sites AB, CD
# AB adjacent, CD adjacent
#------------------------------------------
function get_4pdm(wf::MPS,locs)
    psi = copy(wf)
    A,C = locs

    orthogonalize!(psi,A)
    ket = psi[A]
    for k in A+1:C+1
        ket *= psi[k]
    end
    rho = prime(ket,"Site") * dag(ket)

    if C-A>2
        inds_list = inds(rho)
        trinds1 = collect(3 : C-A)
        trinds2 = trinds1[end] .+ trinds1 .+ 2
        for idx in 1:length(trinds1)
            rho = rho*delta(inds_list[trinds1[idx]],inds_list[trinds2[idx]])
        end
    end
    #-------------------
    return rho
end

get_4pdm (generic function with 1 method)

In [6]:
#   calc_Eigenvalues of density matrix
#-----------------------------------
function calc_Eigenvalues(density_matrix)
    egn_val,_ = eigen(density_matrix,ishermitian=true)
    #-----------------------------------
    return diag(array(egn_val))
end


#   eigenvalue based generic properties of a density matrix
#-----------------------------------
function calc_Norm(density_matrix)
    rho = copy(density_matrix)

    egn_val = calc_Eigenvalues(rho)
    Norm = sum(egn_val)
    Purity = sum(egn_val.^2)

    #-----------------------------------
    return [Norm,Purity]
end

#   eigenvalue based generic properties of a density matrix
#-----------------------------------
function calc_SvN(density_matrix)
    rho = copy(density_matrix)

    egn_val = calc_Eigenvalues(rho)
    SvN = sum( [ - lam*log(lam) for lam in egn_val if lam > 0 ] )

    #-----------------------------------
    return SvN
end


calc_SvN (generic function with 1 method)

In [7]:
#   entanglement details between four sites
#-----------------------------------
function calc_LogNeg_4site(wf::MPS,locs)    
    psi = copy(wf)
    A,C = locs

    rho = get_4pdm(psi,[A,C])
    old = inds(rho)

    function get_LogNeg(rho,old,new)
        rho_PT = swapinds(rho,old,new)
        egn_val = calc_Eigenvalues(rho_PT)
        Neg =  abs(sum( [lam for lam in egn_val if lam<0] ))
        return log(1+2*Neg)
    end
        
    #   ABCD = 1234 
    #   ABCD = 5678

    # wrt A B C D
    new_A  = [old[5],old[2],old[3],old[4],old[1],old[6],old[7],old[8]]
    new_B  = [old[1],old[6],old[3],old[4],old[5],old[2],old[7],old[8]]
    new_C  = [old[1],old[2],old[7],old[4],old[5],old[6],old[3],old[8]]
    new_D  = [old[1],old[2],old[3],old[8],old[5],old[6],old[7],old[4]]

    # wrt AB AC AD
    new_AB = [old[5],old[6],old[3],old[4],old[1],old[2],old[7],old[8]]
    new_AC = [old[5],old[2],old[7],old[4],old[1],old[6],old[3],old[8]]
    new_AD = [old[5],old[2],old[3],old[8],old[1],old[6],old[7],old[4]]

    LogNeg = 1
    for new in [new_A,new_B,new_C,new_D,new_AB,new_AC,new_AD]
        LogNeg *= get_LogNeg(rho,old,new)
    end
    LogNeg = LogNeg^(1.0/7.0)

    #-----------------------------------
    return LogNeg
end

calc_LogNeg_4site (generic function with 1 method)

In [8]:
#define the parameters of the hamiltonian and constuct MPO
#----------------------------------------------------------------------------
J = 1 
B = 0.01

nsweeps = 20
maxdim = [10,25,50,100,150,200,250,300,400,500]
cutoff = [1E-11]

#--------------------
N = 14
#--------------------

site_list = collect(1:N) 
sites = siteinds( k->isodd(k) ? "S=1/2" : "S=3/2", N ) 
#sites = siteinds("S=1/2", N)

H = Hamiltonian(sites) 

# ground state and first excited state
#--------------------------------------------
energy0,psi0 = dmrg(H,random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true) ;
#energy1,psi1 = dmrg(H,[psi0],random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true) ;

After sweep 1 energy=-13.51595126540083  maxlinkdim=8 maxerr=1.07E-15 time=30.953
After sweep 2 energy=-13.805841035102429  maxlinkdim=25 maxerr=2.65E-08 time=0.076
After sweep 3 energy=-13.827901722404903  maxlinkdim=50 maxerr=4.23E-09 time=0.365
After sweep 4 energy=-13.82997174267847  maxlinkdim=100 maxerr=5.35E-10 time=1.093
After sweep 5 energy=-13.831810028847753  maxlinkdim=150 maxerr=6.99E-11 time=1.998
After sweep 6 energy=-13.833470311052688  maxlinkdim=186 maxerr=9.91E-12 time=2.443
After sweep 7 energy=-13.83492091402714  maxlinkdim=180 maxerr=9.88E-12 time=2.199
After sweep 8 energy=-13.836142977095415  maxlinkdim=174 maxerr=9.92E-12 time=1.955
After sweep 9 energy=-13.837107172207944  maxlinkdim=167 maxerr=9.88E-12 time=1.748
After sweep 10 energy=-13.837872409263856  maxlinkdim=159 maxerr=9.92E-12 time=1.738
After sweep 11 energy=-13.83847600614601  maxlinkdim=152 maxerr=9.87E-12 time=1.565
After sweep 12 energy=-13.838949927963704  maxlinkdim=147 maxerr=9.98E-12 time=1.

In [9]:
#define the parameters of the hamiltonian and constuct MPO
#----------------------------------------------------------------------------

J = 1 
B = 0.01

nsweeps = 20
maxdim = [10,25,50,100,150,200,250,300,400,500]
cutoff = [1E-11]

#--------------------
N = 16
#--------------------

site_list = collect(1:N) 

sites = siteinds("S=1/2", N)

H1 = Hamiltonian(sites) 

# ground state and first excited state
#--------------------------------------------
energy0,psi01 = dmrg(H1,random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true) ;
#energy1,psi1 = dmrg(H,[psi0],random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true) ;

After sweep 1 energy=-6.868725440623164  maxlinkdim=4 maxerr=4.34E-16 time=0.032
After sweep 2 energy=-7.100024733919855  maxlinkdim=16 maxerr=3.92E-16 time=0.066
After sweep 3 energy=-7.142286239272012  maxlinkdim=50 maxerr=1.22E-07 time=0.287
After sweep 4 energy=-7.142296354967554  maxlinkdim=100 maxerr=7.55E-10 time=0.680
After sweep 5 energy=-7.142296360408432  maxlinkdim=138 maxerr=9.94E-12 time=0.778
After sweep 6 energy=-7.142296360408429  maxlinkdim=138 maxerr=9.94E-12 time=0.924
After sweep 7 energy=-7.142296360408443  maxlinkdim=138 maxerr=9.94E-12 time=0.768
After sweep 8 energy=-7.142296360408441  maxlinkdim=138 maxerr=9.94E-12 time=0.861
After sweep 9 energy=-7.14229636040843  maxlinkdim=138 maxerr=9.94E-12 time=0.699
After sweep 10 energy=-7.1422963604084355  maxlinkdim=138 maxerr=9.94E-12 time=0.795
After sweep 11 energy=-7.142296360408434  maxlinkdim=138 maxerr=9.94E-12 time=0.723
After sweep 12 energy=-7.142296360408441  maxlinkdim=138 maxerr=9.94E-12 time=0.604
After

In [19]:
using Plots
using LaTeXStrings

seps=0:5
enta= []
enta1= []
A = 1
    println("Mixed spin chain 1/2-3/2")
for sep in seps
    D =  A+2+sep
    ent = calc_LogNeg_4site(psi0,[A,D])
    push!(enta, ent)
    println(f"{sep} \t {ent}")
end
println("----------------------------")
println("----------------------------")
    println("Spin-half")
for sep in seps
    D =  A+2+sep
    ent1 = calc_LogNeg_4site(psi01,[A,D])
    push!(enta1, ent1)
    println(f"{sep} \t {ent1}")
end

plot(seps, enta, xlabel= "Separation", ylabel=L"\mathcal{N}_4", label="1/2--3/2", marker=:circle, color="red", lw=2)
plot!(seps, enta1, xlabel= "Separation", ylabel=L"\mathcal{N}_4",label="1/2--1/2", linestyle=:dash, marker=:circle, color="blue", lw=2)

Mixed spin chain 1/2-3/2
0 	 0.5037433205242009
1 	 0.23615935966105367
2 	 0.1143945892179785
3 	 0.09439343163591109
4 	 0.0939645632344453
5 	 0.09299430727600251
----------------------------
----------------------------
Spin-half
0 	 0.5391934288380282
1 	 0.2764806800870662
2 	 0.0
3 	 0.21169065021482764
4 	 0.0
5 	 0.0


In [11]:
A = 1
D = 4
rho1 = get_4pdm(psi0,[A,D])
d0, u0 = eigen(rho1,ishermitian=true) 
@show diag(array(d0))

diag(array(d0)) = [0.7429674293516053, 0.12111757096227849, 0.05782663836236382, 0.05449796643914865, 0.009425118559787258, 0.008845461275842131, 0.004241343106169278, 0.000685290715900656, 0.00012974967121834338, 8.469228911735569e-5, 3.694227732000561e-5, 3.56494571928513e-5, 3.397554881234956e-5, 2.7705184924371836e-5, 2.590614805211883e-5, 4.788453770037338e-6, 4.658211252520042e-6, 3.436483783801505e-6, 2.941028929359944e-6, 1.6629861520799012e-6, 6.067895732758159e-7, 2.2181323684361828e-7, 1.3520653481839e-7, 2.500176599991823e-8, 2.379346140801538e-8, 2.22804432290982e-8, 1.4063778621322455e-8, 8.77511605932175e-9, 5.817285639364342e-9, 4.306096744098202e-9, 1.8338952783433393e-9, 1.2503620421679196e-9, 1.0326714446843963e-9, 4.795029846210582e-10, 2.980191597447527e-10, 2.222645626827687e-10, 1.530930003492026e-10, 1.0481304015974457e-10, 8.848500686422806e-11, 4.1245656080549355e-11, 2.8119255925748978e-11, 2.405212010541793e-11, 2.0015641054600677e-11, 1.6518764036270053e-11

64-element Vector{Float64}:
 0.7429674293516053
 0.12111757096227849
 0.05782663836236382
 0.05449796643914865
 0.009425118559787258
 0.008845461275842131
 0.004241343106169278
 0.000685290715900656
 0.00012974967121834338
 8.469228911735569e-5
 3.694227732000561e-5
 3.56494571928513e-5
 3.397554881234956e-5
 ⋮
 4.706594887284739e-13
 2.980972833691556e-13
 2.4575393215673605e-13
 8.065486830654093e-14
 7.078581311518974e-14
 4.5979096627691594e-14
 2.406952127550648e-14
 1.9743802477084073e-14
 4.7774896489962434e-15
 3.7569725347294715e-15
 7.210056171785568e-16
 3.0875215683295324e-17